# cell2func

> Fill in a module description here

In [ ]:
#| default_exp core.cell2func

In [ ]:
#| export
from pathlib import Path
import sys
import ast
from IPython import get_ipython
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

from fastcore.all import argnames

In [ ]:
#| export
@magics_class
class Cell2Func(Magics):
    def __init__(self, shell):
        super().__init__(shell)
        self.variable_values = {}
        self.variables = {}
        
    @cell_magic
    def cell2file (self, folder, cell):
        
        folder = Path(folder)
        folder.mkdir(parents=True, exist_ok=True)

        with open(folder / "module.py", "w") as file_handle:
            file_handle.write(cell)

        get_ipython().run_cell(cell)
    
    @cell_magic
    def function (self, func, cell):
        "my cell magic"
        
        get_variables_before = f'\nkeep_variables ("before_{func}", locals ())'
        get_ipython().run_cell(get_variables_before)
        
        cell += f'\nkeep_variables ("{func}", locals ())'
        get_ipython().run_cell(cell)
        
        self.variable_values[func] = {k:self.variable_values[func][k] for k in set(self.variable_values[func]).difference(self.variable_values[f'before_{func}'])}
        
        print (self.variable_values[func])
        
        root = ast.parse (cell)
        self.variables[func] = sorted ({node.id for node in ast.walk(root) if isinstance(node, ast.Name) and not callable(eval(node.id))})
        print (self.variables[func])

def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    magics = Cell2Func(ipython)
    ipython.register_magics(magics)

In [ ]:
#| export
def keep_variables (function, variables, self=None):
    "Store params named in comma-separated `names` from calling context into attrs in `self`"
    frame_number = 1
    while not isinstance (self, Cell2Func):
        fr = sys._getframe(frame_number)
        args = argnames(fr, True)
        if len(args)>0:
            self = fr.f_locals[args[0]]
        frame_number += 1
    variables = {k: variables[k] for k in variables if not k.startswith ('_') and not callable(variables[k])}
    variables_field = getattr(self, 'variable_values')
    variables_field[function] = variables

In [ ]:
load_ipython_extension (get_ipython())

In [ ]:
%%function final
a = 1
b = 2
print (f'a + b is {a+b}')
c = a*b
print (f'a * b is {c}')

a + b is 3
a * b is 2
{}
['a', 'b', 'c']


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()